In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:

class MF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, c_vector=1.0):
        super(MF, self).__init__()
        
        # Embedding layers for users and items
        self.user = nn.Embedding(num_users, embedding_dim)
        self.item = nn.Embedding(num_items, embedding_dim)
        
        # Constant regularization hyperparameter
        self.c_vector = c_vector

        # Initialize weights for the embedding layers
        nn.init.normal_(self.user.weight, std=0.01)
        nn.init.normal_(self.item.weight, std=0.01)

    def forward(self, train_x):
        # Отримання індексів користувачів та товарів
        user_id = train_x[:, 0]  # Індекси користувачів
        item_id = train_x[:, 1]  # Індекси товарів

        # Ініціалізація вектора користувача та товару з використанням їх індексів
        vector_user = self.user(user_id)
        vector_item = self.item(item_id)

        # Обчислення взаємодії користувача та товару: добуток p_u та q_i
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        
        return ui_interaction

    
    def loss(self, prediction, target):
        # Обчислення середньоквадратичної помилки між цільовим значенням та передбаченням
        loss_mse = F.mse_loss(prediction, target.squeeze())

        # Регуляризація L2 для матриць користувачів та товарів
        prior_user = l2_regularize(self.user.weight) * self.c_vector
        prior_item = l2_regularize(self.item.weight) * self.c_vector

        # Сума середньоквадратичної помилки та регуляризації
        total = loss_mse + prior_user + prior_item
        return total

def l2_regularize(tensor):
    # Обчислення L2 регуляризації для тензора
    return torch.norm(tensor, p=2)
